<a href="https://colab.research.google.com/github/airoprojects/vessel-classification/blob/main/code/VesselFormer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q requests wget

  Preparing metadata (setup.py) ... done


In [7]:
import requests

# url = 'https://drive.google.com/file/d/1RfgxSv0HyzFARoznHVTtiwl3IXcnGeZe/view?usp=drive_link'
# filename = 'stead.zip'

# response = requests.get(url, stream=True)
# with open(filename, 'wb') as file:
#     for chunk in response.iter_content(chunk_size=1024):
#         if chunk:  # filter out keep-alive new chunks
#             file.write(chunk)

# !gdown --fuzzy 'https://drive.google.com/file/d/1RfgxSv0HyzFARoznHVTtiwl3IXcnGeZe/view?usp=drive_link'
# !gdown "https://drive.google.com/uc?id=1RfgxSv0HyzFARoznHVTtiwl3IXcnGeZe"

import wget
# https://drive.google.com/file/d/1vB6k-oAJmkZd7LEMc9Myo2XAAq-eJTTJ/view?usp=drive_link
file_id = '1CUgpKZlo9NrA4ILmA1Zna89rveK7M9Ks'
download_url = f'https://drive.google.com/uc?export=download&id={file_id}'
# response = requests.get(download_url)
# print(response.headers.get("Content-Type"))
wget.download(download_url)
# wget.download("https://drive.google.com/uc?export=download?id=1RfgxSv0HyzFARoznHVTtiwl3IXcnGeZe")
# !wget --no-check-certificate 'https://drive.google.com/file/d/1RfgxSv0HyzFARoznHVTtiwl3IXcnGeZe/view?usp=drive_link' -O stead.zip

ValueError: not enough values to unpack (expected 2, got 1)

In [4]:
# !gdown https://drive.google.com/file/d/1CUgpKZlo9NrA4ILmA1Zna89rveK7M9Ks/view?usp=sharing
!gdown  https://drive.google.com/uc?id=1CUgpKZlo9NrA4ILmA1Zna89rveK7M9Ks

Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1CUgpKZlo9NrA4ILmA1Zna89rveK7M9Ks

but Gdown can't. Please check connections and permissions.


NameError: name 'stead' is not defined

In [16]:
import shutil


# Full path of the archive file
zip_file = "./content/stead.zip"

# Format of archive file
archive_format = "zip"

# Unpack the archive file
shutil.unpack_archive(filename, zip_file, archive_format)
print("Archive file unpacked successfully.")

ReadError: stead.zip is not a zip file